In [1]:
"|IMPORT PACKAGES|"
import pandas             as pd
import datetime
from   bokeh.plotting     import show, figure, output_file, save
from   bokeh.io           import show, output_notebook, curdoc, export_png
from   bokeh.models       import ColumnDataSource,LinearAxis, Range1d, NumeralTickFormatter, LabelSet, Label, BoxAnnotation, DatetimeTickFormatter, Legend, Span
from   bokeh.models.tools import HoverTool
from   bokeh.palettes     import Category20
from   bokeh.models       import Arrow, NormalHead, OpenHead, VeeHead
from   datetime           import datetime as dt

In [5]:
"|IMPORT DATA|"
path = r'https://github.com/ncachanosky/research/blob/master/Economic%20Series/'
file = r'Resumen%20Estadistico%20-%20Argentina.xlsx?raw=true'
IO   = path + file

sheet = 'ANUAL'

data = pd.read_excel(IO, sheet_name = sheet, usecols="B,GL:GX", skiprows=2, nrows=27, engine='openpyxl') # Be patient...

data = data.rename(columns={"CUENTA CORRIENTE"   :"CTA_CTE"})
data = data.rename(columns={"CUENTA CAPITAL"     :"CTA_CAP"})
data = data.rename(columns={"CUENTA FINANCIERA"  :"CTA_FIN"})
data = data.rename(columns={"ERRORES Y OMISIONES":"ERRORES"})

In [6]:
"|CHECK DATA|"
data

,YEAR,CTA_CTE,Bienes y Servicios,Bienes,Servicios,Rentas,Transferencias.1,CTA_CAP,CTA_FIN,Inversion extranjera directa,Inversion en cartera,Otras inversiones,ERRORES,RESERVAS
0,1982-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1983-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1984-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1985-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1986-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1987-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1988-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1989-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1990-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1991-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
"|BUILD PLOT|"

data['y1'] = data['CTA_CTE']
data['y2'] = data['CTA_FIN'] + data['CTA_CAP']
data['y3'] = -data['RESERVAS']

cds    = ColumnDataSource(data)

#BUILD FIGURE
p = figure(title        = 'EL HUB ECONÓMICO | BALANZA DE PAGOS',
           x_axis_label = '',
           y_axis_label = 'Millones de dólares',
           x_axis_type  = "datetime",
           plot_height  = 400,
           plot_width   = 700)

p.xaxis[0].formatter = DatetimeTickFormatter(months="%b %Y")
p.toolbar_location = "right"
p.toolbar.autohide = True

#AXIS 1 (LEFT)
p1  = p.line("YEAR", "y1", line_alpha=0.50, width=2, color='blue' , muted_alpha=0.15, source=cds)
p2  = p.line("YEAR", "y2", line_alpha=0.50, width=2, color='red'  , muted_alpha=0.15, source=cds)
p3  = p.line("YEAR", "y3", line_alpha=0.75, width=4, color='black', muted_alpha=0.15, source=cds, legend_label='Reservas')

p.line('YEAR', 'ERRORES', color='black', line_dash='dashed', line_alpha=0.75, muted_alpha=0.15,  source=cds, legend_label='Erroes y omisiones')

p4  = p.varea(x='YEAR', y1=0, y2='y1', fill_color='blue', fill_alpha=0.50, muted_alpha=0.15, source=cds, legend_label='Cuenta corriente')
p5  = p.varea(x='YEAR', y1=0, y2='y2', fill_color='red' , fill_alpha=0.50, muted_alpha=0.15, source=cds, legend_label='Cuenta capital y Cuenta financiera')

p.add_tools(HoverTool(renderers=[p1], tooltips = [("Cta. Corriente"        ,"@y1{$0,0}")], mode="vline"))
p.add_tools(HoverTool(renderers=[p2], tooltips = [("Cta. Cap. y Financiera","@y2{$0,0}")], mode='vline'))
p.add_tools(HoverTool(renderers=[p3], tooltips = [("Reservas"              ,"@y3{$0,0}")], mode='vline'))

#LEGEND
p.yaxis[0].formatter = NumeralTickFormatter(format="$0,0")

h_axis = Span(location=0, dimension="width", line_color="black")
p.add_layout(h_axis)

p.legend.location     = "top_left"
p.legend.orientation  = "horizontal"
p.legend.click_policy = "mute"
show(p)

In [ ]:
"|EXPORT .PNG FILE|"

export_png(p, filename="bop.png")

In [ ]:
"|# CREATE HTML FILE|"

output_file(filename="bop.html", title="Balanza de Pagos")
save(p)

In [8]:
"|CREATE JSON FILE|"
import json
import bokeh.embed

from bokeh.embed import json_item

j = json.dumps(json_item(p, "bop"))
with open("bop.json", "w") as fp:
    json.dump(j, fp)